## 参考資料
- [詳細確率ロボティクスのgit](https://github.com/ryuichiueda/LNPR_BOOK_CODES)

### ベイズフィルタによる自己位置推定
**自己位置推定に使える情報**
- $x_0$:最初の姿勢
- $u_{1:t}$:各時刻の制御指令
- $z_{1:t}$:各時刻のセンサ値
- モデル
  - $p(x|x_{t-1},u_t)$:状態遷移モデル
    - 1個前の姿勢$x_{t-1}$と今回の制御指令$u_t$から状態(姿勢)がどう更新されるかの確率分布
  - $p(z|x)$:観測モデル
    - 現在の姿勢からわかるセンサ値の確率分布？

**計算したい確率分布**
- $p_t(x|x_0,u_{t:1},z_{t:1})$
  - 初期姿勢、これまで与えた制御指令、得られたセンサ値→現在のロボットの姿勢の確率分布を求めたい
  - これを**信念(belief)**とよび、ロボットの信念分布を$b_t(x)$と定義する

**初期信念**
- 初期信念を$b_0と定義$
  - 今回は初期信念が分かっているという前提で、とがった確率分布を考える

**ロボットの移動による信念の更新**
- 時刻t-1からtまでロボットが$u_t$で動くとき、信念分布も$b_{t-1}$から変化
  - $b_{t-1}(x)\rightarrow b_{t-1}(x_0,u_{1:t},z_{1:t-1})=\hat{b_t}$
  - $\hat{b_t}$は$b_t$と違って$z_t$の情報がまだない(zがt:t-1になっていることに注意)
- 得られた信念分布を積分したものが最終的な信念分布になる
  - $x'$は移動前の分布におけるある姿勢
  - $\hat{b_t}=\int_{/{x'\in{\chi}}}p(x|x',u_t)b_{t-1}(x')dx'$
    - 移動前のある姿勢X'の信念分布$b_{t-1}(x')$に、移動後の分布の確率密度$p(x|x',u_t)$をかける。これを移動前のある姿勢x'の全部について積分
    - ポイントは、移動前も姿勢は分布をもっていて(そのうちの一つがx')、移動前の分布に確率密度をかけて積分したものを移動後の分布と考える
  - $\hat{b_t}=<p(x|x',u_t)>_{b_{t-1}(x')}$
    - 移動前のある姿勢の分布$b_{t-1}(x')$に確率密度の期待値をかけたものとも解釈できる
      - どうせ移動前の1つの分布から移動後の1つの分布ができるだけだから、いちいち積分しないで確率密度の期待値としてまとめて、一括で移動後の分布を計算
  - 一般に使われる決まった名前はないが「マルコフ連鎖の式」とか「状態遷移の式」と呼ぶこととする
    - マルコフ性：次の状態は直前の姿勢と制御出力からしか決まらない、それより前の情報は不要

**ランドマークの観測による信念の更新**
- $\hat{b_t}(x)$に$z_t$の情報を追加
  - $\hat{b_t}(x)\rightarrow \hat{b_t}(x|z_t)=b_t(x)$
  - ベイズの定理から、各センサ値の掛け算によって分布を計算できる
    - $b_t(x)=\hat{b_t}(x|z_t)=\eta p(z_t|x) \hat{b_t}(x)$
  - なお、各センサ値の独立性を仮定すると
    - $b_t(x)=\eta \hat{b_t}(x) \prod_{j=0}^{N_m-1} p_j(z_{j, t}|x)$
  - センサ値がない場合は$\hat{b_t}(x)=b_t(x)$

**ベイズフィルタ**
- 以上をまとめると、2つの式で信念の更新ができる
- 移動時：$\hat{b_t}=<p(x|x',u_t)>_{b_{t-1}(x')}$
- 観測時：$b_t(x)=\eta p(z_t|x) \hat{b_t}(x)$
- これをベイズフィルタという
- では、これをどうやって実装するのか？
  - その具体的なやり方を今後扱っていく



### パーティクルフィルタによる自己位置推定
- ベイズフィルタについて、ここではパーティクルフィルタを使って実装する
  - パーティクルの集合で信念分布を表現
  - パーティクル：ロボットの分身だとここでは考える

**移動後のパーティクルの姿勢更新** 
- まずはセンサなしロボットの姿勢の更新を考える
- 前章でやった、ノイズなどをのっけたモデルをMclに実装すれば、ロボットの姿勢が移動によって一意に定まらなくなる様子を表現できる
- が、これはノイズモデルが分かっている前提になるので、実機のロボットだと不可能
  - そこで、ノイズとは別にロボットの動きのバラつきを実装して、その統計から妥当そうなモデルを実装する

**パーティクルを動かすための状態遷移モデル**
- 姿勢のバラつきを単純なガウス分布で表現(あくまで詳解確率ロボティクスであ扱う例)
  - 4章のノイズモデルと違うけど、中心極限定理があるので、ガウス分布だとそこそこ良いだろう
  - ガウス分布を4つの標準偏差で表現する、つまりこれを実験で求める
    - $\sigma_{\nu \nu}$：直線1[m]で生じる道のりのばらつき
      - まっすぐ進んだけど進みすぎたor進み足りなかった
    - $\sigma_{\nu \omega}$：回転1[rad]で生じる道のりのばらつき
      -  回転して中心が少しずれた
    - $\sigma_{\omega \nu}$：直線1[m]で生じるロボットの向きのばらつき
      - まっすぐ進んでいるつもりが曲がってた
    - $\sigma_{\omega \omega}$：回転1[rad]で生じるロボットの向きのばらつき
      - 1回転したはずが、実際は1回転より多いorすくなかった

**速度、角速度に載せる誤差量**
- じゃあ、具体的にばらつきをどれくらい載せますか？という話
- 1mや1radだと単位が違うので単位変換が必要なのが少々面倒
- まずは$\sigma_{\nu \nu}$のときの$\nu$に載せる雑音の量の計算
  - $\delta_{\nu \nu} \sim \mathcal{N}(0, \sigma_{\nu \nu}^{2})$
    - $\delta_{\nu \nu}$：1[m]あたり、つまり距離に乗せる誤差
  - $\delta '_{\nu \nu} =\sqrt{|\nu|/\Delta{t}}$
    - $\delta_{\nu \nu}$：速度に乗せる誤差
      - これは、分散$\sigma$(誤差の二乗)は移動量や回転量に比例すると仮定しているため、ノイズの大きさの期待値($\sigma$の二乗値)も同様に比例する(2章参照)ことから計算
        - $\delta_{\nu \nu}^{2}:(\delta_{\nu \nu}^{\prime} \Delta t)^{2}=1:|\nu| \Delta t$
          - 分散の大きさ(左辺)は移動距離に比例する(右辺)
- これを4種類の全てで定義し、行列でまとめると
  - $\left(\begin{array}{c}\nu^{\prime} \\ \omega^{\prime}\end{array}\right)=\left(\begin{array}{c}\nu \\ \omega\end{array}\right)+\left(\begin{array}{l}\delta_{\nu \nu} \sqrt{|\nu| / \Delta t}+\delta_{\nu \omega} \sqrt{|\omega| / \Delta t} \\ \delta_{\omega \nu} \sqrt{|\nu| / \Delta t}+\delta_{\omega \omega} \sqrt{|\omega| / \Delta t}\end{array}\right)$
  - $(\nu \omega)^{\top}:$ 制御指令
  - $\left(\nu^{\prime} \omega^{\prime}\right)^{\top}$ :実際の速度

In [77]:
# conect to drive on colab
# from google.colab import drive
# drive.mount("/content/drive")
# dir_path="./drive/MyDrive/Colab Notebooks/ProbabilisticRobotics/"
dir_path="./"

import sys
sys.path.append(dir_path)
from robot import *
from scipy.stats import expon,norm,uniform,multivariate_normal

import numpy as np
pi=np.pi

In [82]:
class EstimationAgent(Agent):
    def __init__(self,time_interval,nu,omega,estimator):
        super().__init__(nu,omega)
        self.estimator=estimator
        self.time_interval=time_interval

        self.prev_nu=0.0
        self.prev_omega=0.0
             
    def decision(self,observation=None):
        self.estimator.motion_update(self.prev_nu,self.prev_omega,self.time_interval)
        self.prev_nu,self.prev_omega=self.nu,self.omega
        return self.nu,self.omega
   
    def draw(self,ax,elems):
        self.estimator.draw(ax,elems)
   

In [79]:
class Particle:
    def __init__(self,init_pose):
        self.pose=init_pose
        
    def motion_update(self,nu,omega,time,noise_rate_pdf):
        ns=noise_rate_pdf.rvs()
        noised_nu=nu+ns[0]*np.sqrt(abs(nu)/time)+ns[1]*np.sqrt(abs(omega)/time)
        noised_omega=omega+ns[2]*np.sqrt(abs(nu)/time)+ns[3]*np.sqrt(abs(omega)/time)
        self.pose=IdealRobot.state_transition(noised_nu,noised_omega,time,self.pose)

In [80]:

class Mcl:
    def __init__(self,init_pose,num,motion_noise_stds) -> None:
        self.particles=[Particle(init_pose) for i in range(num)]
        
        # 4次元のガウス分布のオブジェクトを作成
        # diagは対角行列の生成
        v=motion_noise_stds
        c=np.diag([v["nn"]**2,v["no"]**2,v["on"]**2,v["oo"]**2])
        self.motion_noise_rate_pdf=multivariate_normal(cov=c)
 
    def motion_update(self,nu,omega,time):
            for p in self.particles:
                p.motion_update(nu,omega,time,self.motion_noise_rate_pdf)
 
    def draw(self,ax,elems):
        xs=[p.pose[0] for p in self.particles]
        ys=[p.pose[1] for p in self.particles]
        vxs=[np.cos(p.pose[2]) for p in self.particles]
        vys=[np.sin(p.pose[2]) for p in self.particles]
        elems.append(ax.quiver(xs,ys,vxs,vys,color="blue",alpha=0.5))
        
    

In [83]:
motion_noise_stds={"nn":0.01,"no":0.02,"on":0.03,"oo":0.04}
time_interval=0.1
world = World(30, time_interval)

# m = Map()                                  
# m.append_landmark(Landmark(-4,2))
# m.append_landmark(Landmark(2,-3))
# m.append_landmark(Landmark(3,3))
# world.append(m)     

init_pose=np.array([0,0,0]).T
estimator=Mcl(init_pose,100,motion_noise_stds)
circling=EstimationAgent(time_interval,0.2,10/180*pi,estimator)
r= Robot( init_pose, sensor=None, agent=circling,color="red")
world.append(r)

world.draw()

